In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
#노드 클래스 정의
class MerkleTreeNode:
    def __init__(self,value):
        self.left = None #자식노드가 무엇인지(left)
        self.right = None #자식노드가 무엇인지(right)
        self.value = value
        self.hashValue = hashlib.sha256(value.encode('utf-8')).hexdigest()
        #새롭게 추가한 부분 할수있다!
        self.parentNode = None # 부모노드 설정 -> 대체될 수 있을듯?
        self.isRoot = False #루트노드 인지
        self.x = None #노드의 x좌표
        self.y = None #노드의 y좌표
        self.text = '' #노드 텍스트 내용 담기 --> value 와 hashvalue 로 대체?
        self.isLeaf = False # 끝마디 여부
    
    #층수 따지는거 같음?
    def getLevel(self, cnt = 1):
        if self.isRoot:
            return cnt
        else:
            cnt += 1
            cnt = self.parentNode.getLevel(cnt)
            return cnt
    
    #왼쪽 자식 설정
    def setLeftChild(self, node):
        self.left = node
        node.parentNode = self
    
    #오른쪽 자식 설정
    def setRightChild(self, node):
        self.right = node
        node.parentNode = self

In [12]:
class Tree:
    def __init__(self, root):
        assert root.isRoot, 'node should be specified as root'
        self.__root = root
    
    def getRoot(self):
        return self.__root
    
    def getLengthOfBranch(self, node, cnt = 1):
        if node.parentNode is None:
            return cnt
        else:
            cnt += 1
            return self.getLengthOfBranch(node.parentNode, cnt)
            
    def getDepth(self, remove_Leaf=False):
        all_nodes = self.traverseInOrder()
        if remove_Leaf:
            depth = max([self.getLengthOfBranch(node) for node in all_nodes if not node.isLeaf])
        else:
            depth = max([self.getLengthOfBranch(node) for node in all_nodes])
        return depth
        
    def traverseInOrder(self, node=None):
        if node is None:
            node = self.__root
        res = []
        if node.leftChild != None:
            res = res + self.traverseInOrder(node.leftChild)
        res.append(node)
        if node.rightChild != None:
            res = res + self.traverseInOrder(node.rightChild)
        return res
    
    def getRightMostNode(self, node = None, level = None):
        if node is None:
            node = self.__root
        if level is None:
            return [nd for nd in self.traverseInOrder(node)][-1]
        else:
            return [nd for nd in self.traverseInOrder(node) if nd.getLevel()==level][-1]
        
    def getLeftMostNode(self, node = None, level = None):
        if node is None:
            node = self.__root
        if level is None:
            return [nd for nd in self.traverseInOrder(node)][0]
        else:
            return [nd for nd in self.traverseInOrder(node) if nd.getLevel()==level][0]
    
    def getDistanceBetweenSubtrees(self):
        root.leftChild.parentNode = None
        root.rightChild.parentNode = None
        root.leftChild.isRoot = True
        root.rightChild.isRoot = True
        left_subtree = Tree(root.leftChild)
        right_subtree = Tree(root.rightChild)
        if left_subtree.getDepth() == right_subtree.getDepth():
            level = right_subtree.getDepth()
        elif left_subtree.getDepth() > right_subtree.getDepth():
            level = right_subtree.getDepth()
        else:
            level = left_subtree.getDepth()
 
        lrmn = left_subtree.getRightMostNode(level=level)
        rlmn = right_subtree.getLeftMostNode(level=level)
        x_diff = rlmn.x - lrmn.x
        
        root.leftChild.parentNode = root
        root.rightChild.parentNode = root
        root.leftChild.isRoot = False
        root.rightChild.isRoot = False
        return x_diff
    
    def moveTree(self, shift=1):
        for nd in self.traverseInOrder():
            nd.x = nd.x+shift
        return

In [ ]:
def buildTree(leaves,f):
    nodes = []
    for i in leaves:
        nodes.append(MerkleTreeNode(i))

    while len(nodes)!=1:
        temp = []
        for i in range(0,len(nodes),2):
            node1 = nodes[i]
            if i+1 < len(nodes):
                node2 = nodes[i+1]
            else:
                temp.append(nodes[i])
                break
            f.write("Left child : "+ node1.value + " | Hash : " + node1.hashValue +" \n")
            f.write("Right child : "+ node2.value + " | Hash : " + node2.hashValue +" \n")
            print("Left child : "+ node1.value + "\nHash       : " + node1.hashValue +" \n")
            print("Right child : "+ node2.value + "\nHash        : " + node2.hashValue +" \n")
            concatenatedHash = node1.hashValue + node2.hashValue
            parent = MerkleTreeNode(concatenatedHash)
            parent.left = node1
            parent.right = node2
            f.write("Parent(concatenation of "+ node1.value + " and " + node2.value + ") : " +parent.value + " | Hash : " + parent.hashValue +" \n")
            print("Parent : \n[concatenation of "+ node1.value + "\n and              " + node2.value + "] : "+"\nParent Value:\n" +parent.value + " \nHash : " + parent.hashValue +" \n")
            temp.append(parent)
        nodes = temp 
    return nodes[0]